# Instructor Do: Random Forests

In [1]:
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline


ImportError: cannot import name 'Memory' from 'joblib' (unknown location)

## Loading NJ_Data_cleaned Encoded Data

In [ ]:
# Loading data
nj_df = pd.read_csv("NJ_Data_cleaned.csv")
nj_df.head()

In [ ]:
# Encode the data using get_dummies
nj_encoded = pd.get_dummies(cleaned_df)
nj_encoded.head()

In [ ]:
# Define features set
X = nj_encoded.copy()
X.drop("Severity", axis=1, inplace=True)
X.head()

In [ ]:
# Define target vector
y = nj_encoded["Severity"].ravel()
y[:5]

In [ ]:
# Balance out two classes
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

In [ ]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# # Creating StandardScaler instance
# scaler = StandardScaler()

In [ ]:
# # Fitting Standard Scaller
# X_scaler = scaler.fit(X_train)

In [ ]:
# # Scaling data
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

## Fitting the Random Forest Model

In [ ]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, max_depth = 4,random_state=78)

In [ ]:
# Create a random forest classifier
# rf_model_v2 = RandomForestClassifier(n_estimators=500, min_samples_leaf = 5, random_state=78)

In [ ]:
# Fitting the model
rf_model = rf_model.fit(X_train, y_train)

In [ ]:
# Fitting the model
# rf_model_v2 = rf_model_v2.fit(X_train, y_train)

## Making Predictions Using the Random Forest Model

In [ ]:
# Making predictions using the testing data
# predictions = rf_model.predict(X_test)

In [ ]:
# Making predictions using the testing data
predictions_v2 = rf_model_v2.predict(X_test)

## Model Evaluation

In [ ]:
# Calculating the confusion matrix for y_train
predictions_train = rf_model.predict(X_train)

cm = confusion_matrix(y_train, predictions_train)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_train, predictions_train)

In [ ]:
# Displaying results for y_train
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_train, predictions_train))

In [ ]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [ ]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

## Feature Importance

In [ ]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf_model.feature_importances_
# We can sort the features by their importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

In [ ]:
# Visualize the features by importance
importances_df = pd.DataFrame(sorted(zip(rf_model.feature_importances_, X.columns), reverse=True))
importances_df.set_index(importances_df[1], inplace=True)
importances_df.drop(columns=1, inplace=True)
importances_df.rename(columns={0: 'Feature Importances'}, inplace=True)
importances_sorted = importances_df.sort_values(by='Feature Importances')[0:20]
importances_sorted.plot(kind='barh', color='lightgreen', title= 'Features Importances', legend=False)

In [ ]:
nj_df.dtypes

In [ ]:
nj_df.head()

In [ ]:
nj_df.nunique()

In [ ]:
Wind_Direction_count = nj_df['Wind_Direction'].value_counts()
Wind_Direction_count

In [ ]:
Weather_Condition_count = nj_df['Weather_Condition'].value_counts()
Weather_Condition_count

In [ ]:
Weather_Condition_greater_count = Weather_Condition_count[Weather_Condition_count > 1]
print(Weather_Condition_greater_count[:20])